The documents we are querying are related to BABA, AliBaba , recent articles from Seeking Alpha as well as earnings call transcript. 

In [1]:
document_folder = './documents/'

In [4]:
# Simplest RAG index model
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(document_folder).load_data()
simple_rag_index = VectorStoreIndex.from_documents(documents)
rag_query_engine = simple_rag_index.as_query_engine()
response = rag_query_engine.query("How are Alibaba doing?")
print(response)

Alibaba's recent earnings report raised concerns about the company's fundamentals, particularly in China, causing a 6% drop in its stock price. However, the report also showed promise, and Alibaba attempted to attract investors by announcing a $25 billion stock buyback. Despite expectations of a slowdown in China's economy, the company's free cash flow appears to be in good shape. Additionally, Alibaba's buyback could allow it to repurchase approximately 13% of its outstanding shares. Notably, Jack Ma and Joe Tsai, key figures in the company, have recently purchased a significant amount of Alibaba stock.


In [6]:
from llama_index import ServiceContext
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import SimpleGraphStore
from llama_index import KnowledgeGraphIndex
from llama_index.llms import OpenAI


llm = OpenAI(temperature=0, model="gpt-4-turbo-preview")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size_limit=512)

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

kg_index = KnowledgeGraphIndex.from_documents(documents=documents,storage_context=storage_context,service_context=service_context)
kg_query_engine = kg_index.as_query_engine()


In [9]:
# save kg_index as a file 

import pickle
with open('kg_index.pkl', 'wb') as f:
    pickle.dump(kg_index, f)

In [ ]:
from pyvis.network import Network

g = kg_index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("AlibabaGraph.html")


In [ ]:
response = kg_query_engine.query('What are the most divergent opinions on Alibaba?')
print (response)

Alibaba is facing a mix of challenges and strengths. On one hand, the company is dealing with concerns such as a slow post-pandemic recovery in China, skepticism surrounding the Chinese market, and specific operational setbacks like the delayed Cloud unit spin-off. These factors have contributed to Alibaba missing Q3 expectations and experiencing weak growth in its Cloud segment due to slower business spending and uncertainty about chip supplies.

On the other hand, Alibaba remains a financially robust entity, demonstrating significant profitability and generating strong free cash flow. The company announced a substantial $25 billion stock buyback program, indicating confidence in its financial health and a commitment to returning value to shareholders. This move is supported by Alibaba's impressive free cash flow, which reached 56.5 billion Chinese Yuan in a recent quarter, and an annualized run rate of $32 billion. Despite the challenges, Alibaba's actions, such as the stock buyback 

In [12]:
def rag_query(prompt):
    return rag_query_engine.query(prompt)

def kg_query(prompt):
    return kg_query_engine.query(prompt)

In [ ]:
from ipywidgets import GridspecLayout
import ipywidgets as widgets

grid = GridspecLayout(8,6)


submit_button = widgets.Button(
    description='Query',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Press to submit',
    icon='lock',
    layout=widgets.Layout(height='auto', width='auto'),
)

input_box = widgets.Textarea(
    value=None,
    placeholder='Type a question about your documents',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(height='auto', width='auto'),
    rows=4
)

results_box = widgets.Textarea(
    value=None,
    placeholder='Results...',
    description='Answer:',
    disabled=False,
    layout=widgets.Layout(height='auto', width='auto'),
    rows=4
)

# we will populate this later with our list of methods.
dropdown = widgets.Select(
    options=['RAG','KG Rag'],
    value='RAG',
    # rows=10,
    description='Method:',
    disabled=False
)
filter_methods = {'RAG':rag_query,'KG Rag':kg_query}
dropdown.options = (filter_methods.keys())

grid[4:7,5] = dropdown
grid[1:4,:5] = input_box
grid[1:3,5] = submit_button
grid[4:8,:5] = results_box

# anywhere you can now just update the variable and it will live update.
def question(e):
    user_input = input_box.value
    method = filter_methods[dropdown.value]
    results_box.value=method(user_input).response

submit_button.on_click(question)

grid